# Day 15

Ru-roh, the oxygen system failed -- we need to send a repair droid out there to go out and fix the problem. Luckily for us, the droid's program runs on our intcode computer

## Part 1

>What is the fewest number of movement commands required to move the repair droid from its starting position to the location of the oxygen system?

In [1]:
from typing import Tuple, List, Union

import nbimporter

from day_9 import Computer

Importing Jupyter notebook from day_9.ipynb


In [180]:
# We're gonna create a droid class that keeps moving (1: north, 2: south, 3: east, 4: west)
# until it finds the oxygen system (output of 2)
# It will record the state of each point (D: Droid start, .: path, #: wall, OS: oxygen system) as it travels
# until it finds the system

class RepairDroid:
    def __init__(self, program: List[int]):
        self.computer = Computer(program)
        # Initialize with the Droid starting at hypothetical (0 ,0)
        self.map = {(0, 0): "D"}
        # For all the points that we've visited, record all the directions we have tried at that point.
        # Never give the same direction twice at a given point
        self.attempted_directions_dict = {(0, 0): []}
        self.location = (0, 0)
        self.direction_dict = {1: (0, 1), 2: (0, -1), 3: (-1, 0), 4: (1, 0)}
        self.object_dict = {0: "#", 1: ".", 2: "OS"}
        
    def explore(self, direction: int) -> Tuple[int, str]:
        """Given a direction, see if it already exists on the map. It if doesn't
        then run the program to see what's there, and add it to the map. Finally report
        the new location from the given direction.
        """
        x_incr, y_incr = self.direction_dict.get(direction)
        x_coord = self.location[0] + x_incr
        y_coord = self.location[1] + y_incr
        location = (x_coord, y_coord)
        object_at_location = self.map.get(location)
        if not object_at_location:
            self.computer.inputs += [direction]
            self.computer.run_program()
            obj = self.computer.outputs.pop(0)
            object_at_location = self.object_dict.get(obj)
            self.map[location] = object_at_location
        
        return (location, object_at_location)
    
    def choose_direction(self) -> int:
        """Given the current location, figure out all the possible directions
        (i.e. no walls), and then based off historical movements, choose a direction
        to move
        """
        possible_directions = [
            direction for direction in self.direction_dict
            if self.explore(direction)[1] != "#"
        ]
        
        for direction in possible_directions:
            location, _ = self.explore(direction)
            # Lead us to location we haven't been before
            if not self.attempted_directions_dict.get(location):
                self.attempted_directions_dict[location] = []
                self.attempted_directions_dict[self.location] += [direction]
                return direction
        for direction in possible_directions:
            # Lead us in a direction we haven't taken at this location before
            if direction not in self.attempted_directions_dict.get(self.location):
                self.attempted_directions_dict[self.location] += [direction]
                return direction
        for direction in possible_directions:
            # Lead us in a different direction than we took last time from this location
            if direction != self.attempted_directions_dict.get(self.location)[-1]:
                self.attempted_directions_dict[self.location] += [direction]
                return direction
        
        # If all else fails, go back the way we came before -- prob won't work?
        return possible_directions[0]

    def find_oxygen_sensor(self):
        while self.map[self.location] != "OS":
            direction = self.choose_direction()
            self.location, _ = self.explore(direction)
        return self.location

In [181]:
# Puzzle input
program = [3,1033,1008,1033,1,1032,1005,1032,31,1008,1033,2,1032,1005,1032,58,1008,1033,3,1032,1005,1032,81,1008,1033,4,1032,1005,1032,104,99,1002,1034,1,1039,1002,1036,1,1041,1001,1035,-1,1040,1008,1038,0,1043,102,-1,1043,1032,1,1037,1032,1042,1106,0,124,1001,1034,0,1039,1002,1036,1,1041,1001,1035,1,1040,1008,1038,0,1043,1,1037,1038,1042,1106,0,124,1001,1034,-1,1039,1008,1036,0,1041,1001,1035,0,1040,102,1,1038,1043,1002,1037,1,1042,1106,0,124,1001,1034,1,1039,1008,1036,0,1041,102,1,1035,1040,1001,1038,0,1043,1002,1037,1,1042,1006,1039,217,1006,1040,217,1008,1039,40,1032,1005,1032,217,1008,1040,40,1032,1005,1032,217,1008,1039,5,1032,1006,1032,165,1008,1040,35,1032,1006,1032,165,1102,1,2,1044,1106,0,224,2,1041,1043,1032,1006,1032,179,1102,1,1,1044,1106,0,224,1,1041,1043,1032,1006,1032,217,1,1042,1043,1032,1001,1032,-1,1032,1002,1032,39,1032,1,1032,1039,1032,101,-1,1032,1032,101,252,1032,211,1007,0,38,1044,1106,0,224,1101,0,0,1044,1106,0,224,1006,1044,247,1001,1039,0,1034,1001,1040,0,1035,101,0,1041,1036,102,1,1043,1038,1002,1042,1,1037,4,1044,1106,0,0,4,26,16,55,25,8,4,99,2,21,20,20,56,26,97,81,12,2,4,9,32,7,49,54,5,18,81,16,7,88,4,23,30,66,17,31,27,29,34,26,81,62,27,81,41,84,12,53,90,79,37,22,45,27,17,39,76,1,55,58,44,20,18,57,57,20,76,47,20,44,88,26,43,36,79,12,68,30,19,71,27,21,18,75,18,9,56,29,15,84,8,74,93,1,35,91,39,32,86,9,97,54,4,22,59,13,61,31,19,97,26,82,35,73,23,77,71,59,26,76,78,73,34,85,67,26,1,66,91,79,26,95,5,75,99,29,14,23,26,8,66,97,55,21,25,49,17,99,71,37,62,21,45,46,13,29,30,24,31,63,99,12,12,63,10,64,2,76,3,8,37,94,33,12,47,65,35,65,60,12,88,8,10,49,36,12,14,4,43,82,19,16,51,52,20,17,43,18,33,49,19,93,49,29,86,10,31,92,90,44,26,97,8,63,70,81,28,17,80,23,22,79,56,33,67,61,91,37,4,83,77,16,6,8,33,66,92,46,8,34,23,81,3,93,14,23,72,20,91,16,62,79,7,27,81,10,11,44,65,24,66,77,31,12,53,15,50,84,24,70,29,62,50,5,3,88,13,52,85,42,4,15,39,82,65,18,15,58,37,71,10,13,90,98,29,59,52,3,22,13,59,91,29,23,79,1,7,24,80,79,37,31,77,17,11,64,10,9,8,74,97,6,74,35,73,44,68,29,97,3,45,73,30,28,80,9,48,73,76,7,3,77,83,8,12,41,62,44,10,21,27,74,32,95,73,4,47,71,6,67,17,57,10,67,5,25,74,18,24,57,7,61,66,4,51,14,7,44,29,79,74,11,6,49,75,32,3,98,89,63,5,15,5,74,78,37,7,77,3,13,47,9,33,76,22,47,6,72,12,35,75,39,25,87,83,37,19,91,25,45,22,30,54,83,74,22,71,19,3,3,85,74,37,95,26,67,46,10,12,96,44,50,32,90,3,28,56,24,43,4,1,65,5,9,50,22,44,88,9,48,59,21,24,54,11,35,53,28,7,82,32,24,17,45,88,34,72,95,17,9,39,29,4,55,66,95,22,62,15,71,11,39,51,37,86,49,20,10,63,31,66,59,15,55,93,3,11,28,54,30,41,20,92,7,3,12,54,49,14,33,56,89,21,26,67,20,93,7,64,3,31,60,23,51,36,30,57,20,14,28,88,4,6,69,33,65,98,35,96,80,49,25,68,78,97,30,63,35,73,89,32,64,69,10,68,96,19,89,71,41,32,31,30,90,5,71,20,53,36,51,23,87,19,25,15,34,15,48,19,25,33,14,50,64,11,96,19,34,14,44,33,29,40,16,50,90,22,34,44,17,64,63,18,86,57,29,44,22,98,16,41,20,99,34,14,51,11,4,84,91,66,27,49,6,58,34,95,62,6,45,53,27,72,4,12,40,43,17,41,93,27,30,70,31,47,87,26,64,9,63,59,73,9,11,97,35,56,73,23,58,9,49,13,88,1,87,13,54,21,94,13,69,16,39,2,10,64,13,10,19,96,2,23,1,60,99,47,12,61,37,13,70,24,48,91,7,33,51,10,25,88,33,69,29,98,16,16,60,5,29,44,17,21,41,62,65,8,61,84,27,42,78,72,23,98,16,76,98,77,37,19,49,37,93,83,97,1,63,9,63,27,66,34,74,87,58,3,90,4,48,51,67,32,66,9,56,9,44,1,67,24,49,29,58,20,70,32,73,27,82,0,0,21,21,1,10,1,0,0,0,0,0,0]

In [182]:
droid = RepairDroid(program)

In [183]:
droid.find_oxygen_sensor()

KeyboardInterrupt: 

In [209]:
[direction for direction in droid.attempted_directions_dict if len(droid.attempted_directions_dict[direction]) > 10]

[(-14, -6), (-14, -5), (-14, -7), (-14, -8), (-14, -9), (-15, -9), (-15, -8)]

In [204]:
droid.location = (-15, -9)

In [205]:
possible_directions = [
    direction for direction in droid.direction_dict
    if droid.explore(direction)[1] != "#"
]

possible_directions

[1, 3, 4]

In [206]:
location = droid.explore(1)[0]
print(location)
droid.attempted_directions_dict.get(location, "never seen it")


(-15, -8)


[2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,


In [207]:
droid.location = (-15, -8)

possible_directions = [
    direction for direction in droid.direction_dict
    if droid.explore(direction)[1] != "#"
]

possible_directions

[2, 4]

{(-16, -9): [4],
 (-15, -9): [1,
  3,
  4,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  

In [186]:
droid.map

{(-17, -9): '#',
 (-16, -10): '#',
 (-16, -9): '.',
 (-16, -8): '#',
 (-15, -10): '#',
 (-15, -9): '.',
 (-15, -8): '.',
 (-15, -7): '#',
 (-15, -6): '#',
 (-15, -5): '#',
 (-15, -4): '#',
 (-14, -10): '#',
 (-14, -9): '.',
 (-14, -8): '.',
 (-14, -7): '.',
 (-14, -6): '.',
 (-14, -5): '.',
 (-14, -4): '.',
 (-14, -3): '#',
 (-14, -2): '#',
 (-14, -1): '#',
 (-14, 0): '#',
 (-13, -10): '#',
 (-13, -9): '.',
 (-13, -8): '#',
 (-13, -7): '#',
 (-13, -6): '.',
 (-13, -5): '.',
 (-13, -4): '#',
 (-13, -3): '.',
 (-13, -2): '.',
 (-13, -1): '.',
 (-13, 0): '.',
 (-13, 1): '#',
 (-12, -10): '#',
 (-12, -9): '.',
 (-12, -8): '.',
 (-12, -7): '.',
 (-12, -6): '.',
 (-12, -5): '.',
 (-12, -4): '#',
 (-12, -3): '#',
 (-12, -2): '.',
 (-12, -1): '.',
 (-12, 0): '#',
 (-11, -9): '#',
 (-11, -8): '.',
 (-11, -7): '#',
 (-11, -6): '.',
 (-11, -5): '.',
 (-11, -4): '.',
 (-11, -3): '.',
 (-11, -2): '.',
 (-11, -1): '.',
 (-11, 0): '#',
 (-10, -9): '#',
 (-10, -8): '.',
 (-10, -7): '#',
 (-10, -6): '#

In [165]:
print("map before", droid.map)
print("location", droid.location)
print("attempted directions", droid.attempted_directions_dict)
print("chosen direction", droid.choose_direction())
print("map after", droid.map)

map before {(0, 0): 'D', (0, 1): '#', (0, -1): '#', (-1, 0): '.', (1, 0): '.', (-1, 1): '#', (-1, -1): '#', (-2, 0): '.', (-2, 1): '#', (-2, -1): '#', (-3, 0): '.', (-3, 1): '#', (-3, -1): '.', (-4, 0): '#'}
location (-3, -1)
attempted directions {(0, 0): [3], (-1, 0): [3], (-2, 0): [3], (-3, 0): [2], (-3, -1): []}
chosen direction 2
map after {(0, 0): 'D', (0, 1): '#', (0, -1): '#', (-1, 0): '.', (1, 0): '.', (-1, 1): '#', (-1, -1): '#', (-2, 0): '.', (-2, 1): '#', (-2, -1): '#', (-3, 0): '.', (-3, 1): '#', (-3, -1): '.', (-4, 0): '#', (-3, -2): '.', (-4, -1): '#'}


In [164]:
droid.location = (-3, -1)

In [152]:
droid.explore(1)

((0, 1), '#')